# Домашнее задание

Загрузка необходимых библиотек и данных

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

import warnings
warnings.simplefilter('ignore')

In [ ]:
data = pd.read_csv('data/retail_train.csv')
item_features = pd.read_csv('data/product.csv')
user_features = pd.read_csv('data/hh_demographic.csv')

---

In [ ]:
# Process features dataset

ITEM_COL = 'item_id'
USER_COL = 'user_id'

# Column Processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

In [ ]:
# Split for train, eval, test

# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k
# от размера датасета)


VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

# берем данные для тренировки matching модели
data_train_matcher = data[data['week_no'] < data['week_no'].max() - \
                          (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

# берем данные для валидации matching модели
data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - \
                         (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &  \
                         (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]


# берем данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()  # Для наглядности.
                                             # Далее мы добавим изменения, и они будут отличаться

# берем данные для теста ranking, matching модели
data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]

In [ ]:
# Функция для print data

def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} \
            Items: {df_data[ITEM_COL].nunique()}")

In [ ]:
# Проверка

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

In [ ]:
# Prefilter items (оставляем 5000)

n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, \
                                     item_features=item_features, take_n_popular=5000)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

In [ ]:
# Ищем общих пользователей

common_users = data_train_matcher.user_id.values

data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

print_stats_data(data_train_matcher,'train_matcher')
print_stats_data(data_val_matcher,'val_matcher')
print_stats_data(data_train_ranker,'train_ranker')
print_stats_data(data_val_ranker,'val_ranker')

In [ ]:
common_users

In [ ]:
# Init/train recommender

recommender = MainRecommender(data_train_matcher)
recommender

In [ ]:
# Топ покупок каждого юзера

recommender.top_purchases.head(3)

In [ ]:
# Топ покупок по всему датасету

recommender.overall_top_purchases[:3]

---

### Задание 1

In [ ]:
ACTUAL_COL = 'actual'

result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

In [ ]:
# Рекомендуем топ-N товаров
# N = Neighbors

N_PREDICT = 50

In [ ]:
%%time

result_eval_matcher['own_rec'] = result_eval_matcher[USER_COL]. \
        apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

result_eval_matcher['sim_item_rec'] = result_eval_matcher[USER_COL]. \
        apply(lambda x: recommender.get_similar_items_recommendation(x, N=50))

result_eval_matcher['als_rec'] = result_eval_matcher[USER_COL]. \
        apply(lambda x: recommender.get_als_recommendations(x, N=50))

In [ ]:
%%time
result_eval_matcher['sim_user_rec'] = result_eval_matcher[USER_COL]. \
        apply(lambda x: recommender.get_similar_users_recommendation(x, N=50))

In [ ]:
# Оборачивание

def evalRecall(df_result, target_col_name, recommend_model):
    result_col_name = 'result'
    df_result[result_col_name] = df_result[target_col_name]. \
            apply(lambda x: recommend_model(x, N=25))
    return df_result.apply(lambda row: recall_at_k(row[result_col_name], \
                                                row[ACTUAL_COL], k=N_PREDICT), axis=1).mean()

# Проверка

evalRecall(result_eval_matcher, USER_COL, recommender.get_own_recommendations)

In [ ]:
def calc_recall(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: \
                                      recall_at_k(row[col_name], 
                                                  row[ACTUAL_COL], k=top_k), axis=1).mean()
        
def calc_precision(df_data, top_k):
    for col_name in df_data.columns[2:]:
        yield col_name, df_data.apply(lambda row: \
                                      precision_at_k(row[col_name], 
                                                     row[ACTUAL_COL], k=top_k), axis=1).mean()

#### Recall @ 50

In [ ]:
TOPK_RECALL = 50

sorted(calc_recall(result_eval_matcher, TOPK_RECALL), key=lambda x: x[1],reverse=True)

#### Precision @ 5

In [ ]:
TOPK_PRECISION = 5

sorted(calc_precision(result_eval_matcher, TOPK_PRECISION), key=lambda x: x[1],reverse=True)

*Вывод:* Own recommendtions + top-popular дают лучший recall

---

### Задание 2

In [ ]:
# Подготовка фичей для обучения

df_ranker_train = df_ranker_train.merge(mean_user_checks, on='user_id', how='left')
df_ranker_train = df_ranker_train.merge(mean_user_week_checks, on=['user_id'], how='left')

df_ranker_train = df_ranker_train.merge(mean_basket_item_checks, on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(item_priсes, on=['item_id'], how='left')

df_ranker_train = df_ranker_train.merge(user_item_week_ratios, on=['item_id', 'user_id'], how='left')
df_ranker_train = df_ranker_train.merge(user_item_week_check, on=['user_id', 'item_id'], how='left')

df_ranker_train.head(2)

In [ ]:
# df_ranker_train.to_csv('../data/df_ranker_train.csv', chunksize=100000)

# df_ranker_train = pd.read_csv('../data/df_ranker_train.csv')
# df_ranker_train.head(2)

In [ ]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

In [ ]:
# Обучение модели ранжирования

lgb = LGBMClassifier(objective='binary',
                     max_depth=8,
                     n_estimators=300,
                     learning_rate=0.05,
                     categorical_column=cat_feats)

lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)

In [ ]:
df_ranker_predict = df_ranker_train.copy()

df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

In [ ]:
# Evaluation on test

result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]

result_eval_ranker['own_rec'] = result_eval_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True)

In [ ]:
# Evaluation re-ranked on test

def rerank(user_id):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id].sort_values('proba_item_purchase', ascending=False).head(5).item_id.tolist()

result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

print(*sorted(calc_precision(result_eval_ranker, TOPK_PRECISION), key=lambda x: x[1], reverse=True), sep='\n')

*Вывод:* После добавления по 2 фичи для юзера, товара и пары юзер-товар значение precision@5 двухуровневой модели не выросло по сравнению с precision@5 модели 1-ого уровня